In [16]:
# Import libraries
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer
import nltk 
import string
import re
import numpy as np 
import pandas as pd 
import os
import pandas as pd
import numpy as np
import keras
import pickle

from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout
from sklearn.preprocessing import LabelBinarizer
import sklearn.datasets as skds
from pathlib import Path
import keras.optimizers

import keras
import tensorflow as tf

from tensorflow.python.client import device_lib

pd.set_option('display.max_colwidth', 100)

In [17]:
tweet_df = pd.read_excel("train-20190402-08_55_19.xlsx")
tweet_df.head()

,Id,username,party,text,created_at,retweet_count,favorite_count
0,0,Jaumeasens,bcomu,Que el Suprem no deixi comparèixer davant el Parlament als presos polítics és una altra ofensa a...,2019-01-22 11:09:09,205,378
1,1,subirats9,bcomu,Un artículo a leer de @ainagallego sobre un tema importante “El cambio tecnológico en el trabajo...,2018-12-12 10:20:00,2,2
2,2,subirats9,bcomu,El sentiment de desprotecció davant les incerteses que genera el canvi d’època i les polítiques ...,2018-11-24 23:06:50,20,58
3,3,neusmunte,jxcat,"I cap d’ells te traça amb els micròfons i les grabadores, ni res a veure amb les clavegueres, ve...",2018-10-23 20:13:25,161,462
4,4,cuevillasjacs,jxcat,El @sergimiquel s’ha acomiadat avui com a Secretari General de la @JNCatalunya amb un gran discu...,2019-03-17 18:45:50,58,238


In [18]:
print('Dataset size:',tweet_df.shape)
print('Columns are:',tweet_df.columns)

Dataset size: (7219, 7)
Columns are: Index(['Id', 'username', 'party', 'text', 'created_at', 'retweet_count',
       'favorite_count'],
      dtype='object')


In [19]:
df  = pd.DataFrame(tweet_df[['party', 'text']])
print(set(tweet_df.party.values))

{'psc', 'pp', 'jxcat', 'cup', 'erc', 'bcomu', 'vallsbcn', 'bcncapital'}


In [20]:
def remove_punct(text):
    text  = "".join([char for char in text if char not in string.punctuation])
    text = re.sub('[0-9]+', '', text)
    return text

df['text_punct'] = df['text'].apply(lambda x: remove_punct(x))
df.head(10)

,party,text,text_punct
0,bcomu,Que el Suprem no deixi comparèixer davant el Parlament als presos polítics és una altra ofensa a...,Que el Suprem no deixi comparèixer davant el Parlament als presos polítics és una altra ofensa a...
1,bcomu,Un artículo a leer de @ainagallego sobre un tema importante “El cambio tecnológico en el trabajo...,Un artículo a leer de ainagallego sobre un tema importante “El cambio tecnológico en el trabajo ...
2,bcomu,El sentiment de desprotecció davant les incerteses que genera el canvi d’època i les polítiques ...,El sentiment de desprotecció davant les incerteses que genera el canvi d’època i les polítiques ...
3,jxcat,"I cap d’ells te traça amb els micròfons i les grabadores, ni res a veure amb les clavegueres, ve...",I cap d’ells te traça amb els micròfons i les grabadores ni res a veure amb les clavegueres ves ...
4,jxcat,El @sergimiquel s’ha acomiadat avui com a Secretari General de la @JNCatalunya amb un gran discu...,El sergimiquel s’ha acomiadat avui com a Secretari General de la JNCatalunya amb un gran discurs...
5,psc,"En un món connectat les ciutats són centres de comerç, creixement, innovació, tecnologia i finan...",En un món connectat les ciutats són centres de comerç creixement innovació tecnologia i finances
6,erc,Moltes gràcies @AntoniBassas per tenir-nos sempre presents i no callar per denunciar aquesta inj...,Moltes gràcies AntoniBassas per tenirnos sempre presents i no callar per denunciar aquesta injus...
7,psc,"Escoltant l'amic i mestre Antón Costas a la presentació de ""El final del desconcierto"" al @CdEco...",Escoltant lamic i mestre Antón Costas a la presentació de El final del desconcierto al CdEconomi...
8,psc,#Recomano nou vídeo testimonial del Llegat Pasqual Maragall #LlegatPM @catalunyaeuropa En aquest...,Recomano nou vídeo testimonial del Llegat Pasqual Maragall LlegatPM catalunyaeuropa En aquesta o...
9,pp,"Entrega anual de premios Rosa de Plata de @BarnaCentre , con el candidato @josebouvila y nuestra...",Entrega anual de premios Rosa de Plata de BarnaCentre con el candidato josebouvila y nuestra co...


In [21]:
def tokenization(text):
    text = re.split('\W+', text)
    return text

df['text_tokenized'] = df['text_punct'].apply(lambda x: tokenization(x.lower()))
df.head()

,party,text,text_punct,text_tokenized
0,bcomu,Que el Suprem no deixi comparèixer davant el Parlament als presos polítics és una altra ofensa a...,Que el Suprem no deixi comparèixer davant el Parlament als presos polítics és una altra ofensa a...,"[que, el, suprem, no, deixi, comparèixer, davant, el, parlament, als, presos, polítics, és, una,..."
1,bcomu,Un artículo a leer de @ainagallego sobre un tema importante “El cambio tecnológico en el trabajo...,Un artículo a leer de ainagallego sobre un tema importante “El cambio tecnológico en el trabajo ...,"[un, artículo, a, leer, de, ainagallego, sobre, un, tema, importante, el, cambio, tecnológico, e..."
2,bcomu,El sentiment de desprotecció davant les incerteses que genera el canvi d’època i les polítiques ...,El sentiment de desprotecció davant les incerteses que genera el canvi d’època i les polítiques ...,"[el, sentiment, de, desprotecció, davant, les, incerteses, que, genera, el, canvi, d, època, i, ..."
3,jxcat,"I cap d’ells te traça amb els micròfons i les grabadores, ni res a veure amb les clavegueres, ve...",I cap d’ells te traça amb els micròfons i les grabadores ni res a veure amb les clavegueres ves ...,"[i, cap, d, ells, te, traça, amb, els, micròfons, i, les, grabadores, ni, res, a, veure, amb, le..."
4,jxcat,El @sergimiquel s’ha acomiadat avui com a Secretari General de la @JNCatalunya amb un gran discu...,El sergimiquel s’ha acomiadat avui com a Secretari General de la JNCatalunya amb un gran discurs...,"[el, sergimiquel, s, ha, acomiadat, avui, com, a, secretari, general, de, la, jncatalunya, amb, ..."


In [22]:
stopword = nltk.corpus.stopwords.words('spanish')
stopword.extend(nltk.corpus.stopwords.words('catalan'))
stopword.extend(["als","un","dels","s","l","d","t"])
print(stopword)
def remove_stopwords(text):
    text = [word for word in text if word not in stopword]
    return text
    
df['text_nonstop'] = df['text_tokenized'].apply(lambda x: remove_stopwords(x))
df.head(10)

['de', 'la', 'que', 'el', 'en', 'y', 'a', 'los', 'del', 'se', 'las', 'por', 'un', 'para', 'con', 'no', 'una', 'su', 'al', 'lo', 'como', 'más', 'pero', 'sus', 'le', 'ya', 'o', 'este', 'sí', 'porque', 'esta', 'entre', 'cuando', 'muy', 'sin', 'sobre', 'también', 'me', 'hasta', 'hay', 'donde', 'quien', 'desde', 'todo', 'nos', 'durante', 'todos', 'uno', 'les', 'ni', 'contra', 'otros', 'ese', 'eso', 'ante', 'ellos', 'e', 'esto', 'mí', 'antes', 'algunos', 'qué', 'unos', 'yo', 'otro', 'otras', 'otra', 'él', 'tanto', 'esa', 'estos', 'mucho', 'quienes', 'nada', 'muchos', 'cual', 'poco', 'ella', 'estar', 'estas', 'algunas', 'algo', 'nosotros', 'mi', 'mis', 'tú', 'te', 'ti', 'tu', 'tus', 'ellas', 'nosotras', 'vosostros', 'vosostras', 'os', 'mío', 'mía', 'míos', 'mías', 'tuyo', 'tuya', 'tuyos', 'tuyas', 'suyo', 'suya', 'suyos', 'suyas', 'nuestro', 'nuestra', 'nuestros', 'nuestras', 'vuestro', 'vuestra', 'vuestros', 'vuestras', 'esos', 'esas', 'estoy', 'estás', 'está', 'estamos', 'estáis', 'están', 

,party,text,text_punct,text_tokenized,text_nonstop
0,bcomu,Que el Suprem no deixi comparèixer davant el Parlament als presos polítics és una altra ofensa a...,Que el Suprem no deixi comparèixer davant el Parlament als presos polítics és una altra ofensa a...,"[que, el, suprem, no, deixi, comparèixer, davant, el, parlament, als, presos, polítics, és, una,...","[suprem, deixi, comparèixer, davant, parlament, presos, polítics, altra, ofensa, ciutadania, cat..."
1,bcomu,Un artículo a leer de @ainagallego sobre un tema importante “El cambio tecnológico en el trabajo...,Un artículo a leer de ainagallego sobre un tema importante “El cambio tecnológico en el trabajo ...,"[un, artículo, a, leer, de, ainagallego, sobre, un, tema, importante, el, cambio, tecnológico, e...","[artículo, leer, ainagallego, tema, importante, cambio, tecnológico, trabajo, amenaza, u, oportu..."
2,bcomu,El sentiment de desprotecció davant les incerteses que genera el canvi d’època i les polítiques ...,El sentiment de desprotecció davant les incerteses que genera el canvi d’època i les polítiques ...,"[el, sentiment, de, desprotecció, davant, les, incerteses, que, genera, el, canvi, d, època, i, ...","[sentiment, desprotecció, davant, incerteses, genera, canvi, època, polítiques, neoliberals, pro..."
3,jxcat,"I cap d’ells te traça amb els micròfons i les grabadores, ni res a veure amb les clavegueres, ve...",I cap d’ells te traça amb els micròfons i les grabadores ni res a veure amb les clavegueres ves ...,"[i, cap, d, ells, te, traça, amb, els, micròfons, i, les, grabadores, ni, res, a, veure, amb, le...","[cap, traça, micròfons, grabadores, res, veure, clavegueres, ves, més, em, representen, exiliats..."
4,jxcat,El @sergimiquel s’ha acomiadat avui com a Secretari General de la @JNCatalunya amb un gran discu...,El sergimiquel s’ha acomiadat avui com a Secretari General de la JNCatalunya amb un gran discurs...,"[el, sergimiquel, s, ha, acomiadat, avui, com, a, secretari, general, de, la, jncatalunya, amb, ...","[sergimiquel, acomiadat, avui, secretari, general, jncatalunya, gran, discurs, llargament, ovaci..."
5,psc,"En un món connectat les ciutats són centres de comerç, creixement, innovació, tecnologia i finan...",En un món connectat les ciutats són centres de comerç creixement innovació tecnologia i finances,"[en, un, món, connectat, les, ciutats, són, centres, de, comerç, creixement, innovació, tecnolog...","[món, connectat, ciutats, són, centres, comerç, creixement, innovació, tecnologia, finances, ]"
6,erc,Moltes gràcies @AntoniBassas per tenir-nos sempre presents i no callar per denunciar aquesta inj...,Moltes gràcies AntoniBassas per tenirnos sempre presents i no callar per denunciar aquesta injus...,"[moltes, gràcies, antonibassas, per, tenirnos, sempre, presents, i, no, callar, per, denunciar, ...","[moltes, gràcies, antonibassas, tenirnos, sempre, presents, callar, denunciar, aquesta, injustíc..."
7,psc,"Escoltant l'amic i mestre Antón Costas a la presentació de ""El final del desconcierto"" al @CdEco...",Escoltant lamic i mestre Antón Costas a la presentació de El final del desconcierto al CdEconomi...,"[escoltant, lamic, i, mestre, antón, costas, a, la, presentació, de, el, final, del, desconciert...","[escoltant, lamic, mestre, antón, costas, presentació, final, desconcierto, cdeconomia, brillant..."
8,psc,#Recomano nou vídeo testimonial del Llegat Pasqual Maragall #LlegatPM @catalunyaeuropa En aquest...,Recomano nou vídeo testimonial del Llegat Pasqual Maragall LlegatPM catalunyaeuropa En aquesta o...,"[recomano, nou, vídeo, testimonial, del, llegat, pasqual, maragall, llegatpm, catalunyaeuropa, e...","[recomano, nou, vídeo, testimonial, llegat, pasqual, maragall, llegatpm, catalunyaeuropa, aquest..."
9,pp,"Entrega anual de premios Rosa de Plata de @BarnaCentre , con el candidato @josebouvila y nuestra...",Entrega anual de premios Rosa de Plata de BarnaCentre con el candidato josebouvila y nuestra co...,"[entrega, anual, de, premios, rosa, de, pl

In [23]:
ps = nltk.PorterStemmer()

def stemming(text):
    text = [ps.stem(word) for word in text]
    return text

df['text_stemmed'] = df['text_nonstop'].apply(lambda x: stemming(x))
df.head()

,party,text,text_punct,text_tokenized,text_nonstop,text_stemmed
0,bcomu,Que el Suprem no deixi comparèixer davant el Parlament als presos polítics és una altra ofensa a...,Que el Suprem no deixi comparèixer davant el Parlament als presos polítics és una altra ofensa a...,"[que, el, suprem, no, deixi, comparèixer, davant, el, parlament, als, presos, polítics, és, una,...","[suprem, deixi, comparèixer, davant, parlament, presos, polítics, altra, ofensa, ciutadania, cat...","[suprem, deixi, comparèix, davant, parlament, preso, polític, altra, ofensa, ciutadania, catalun..."
1,bcomu,Un artículo a leer de @ainagallego sobre un tema importante “El cambio tecnológico en el trabajo...,Un artículo a leer de ainagallego sobre un tema importante “El cambio tecnológico en el trabajo ...,"[un, artículo, a, leer, de, ainagallego, sobre, un, tema, importante, el, cambio, tecnológico, e...","[artículo, leer, ainagallego, tema, importante, cambio, tecnológico, trabajo, amenaza, u, oportu...","[artículo, leer, ainagallego, tema, important, cambio, tecnológico, trabajo, amenaza, u, oportun..."
2,bcomu,El sentiment de desprotecció davant les incerteses que genera el canvi d’època i les polítiques ...,El sentiment de desprotecció davant les incerteses que genera el canvi d’època i les polítiques ...,"[el, sentiment, de, desprotecció, davant, les, incerteses, que, genera, el, canvi, d, època, i, ...","[sentiment, desprotecció, davant, incerteses, genera, canvi, època, polítiques, neoliberals, pro...","[sentiment, desprotecció, davant, incertes, genera, canvi, època, polítiqu, neoliber, provoca, a..."
3,jxcat,"I cap d’ells te traça amb els micròfons i les grabadores, ni res a veure amb les clavegueres, ve...",I cap d’ells te traça amb els micròfons i les grabadores ni res a veure amb les clavegueres ves ...,"[i, cap, d, ells, te, traça, amb, els, micròfons, i, les, grabadores, ni, res, a, veure, amb, le...","[cap, traça, micròfons, grabadores, res, veure, clavegueres, ves, més, em, representen, exiliats...","[cap, traça, micròfon, grabador, re, veur, claveguer, ve, mé, em, representen, exiliat, preso, p..."
4,jxcat,El @sergimiquel s’ha acomiadat avui com a Secretari General de la @JNCatalunya amb un gran discu...,El sergimiquel s’ha acomiadat avui com a Secretari General de la JNCatalunya amb un gran discurs...,"[el, sergimiquel, s, ha, acomiadat, avui, com, a, secretari, general, de, la, jncatalunya, amb, ...","[sergimiquel, acomiadat, avui, secretari, general, jncatalunya, gran, discurs, llargament, ovaci...","[sergimiquel, acomiadat, avui, secretari, gener, jncatalunya, gran, discur, llargament, ovaciona..."


In [24]:
wn = nltk.WordNetLemmatizer()
def lemmatizer(text):
    text = [wn.lemmatize(word) for word in text]
    return text

df['text_lemmatized'] = df['text_stemmed'].apply(lambda x: lemmatizer(x))
df.head()

,party,text,text_punct,text_tokenized,text_nonstop,text_stemmed,text_lemmatized
0,bcomu,Que el Suprem no deixi comparèixer davant el Parlament als presos polítics és una altra ofensa a...,Que el Suprem no deixi comparèixer davant el Parlament als presos polítics és una altra ofensa a...,"[que, el, suprem, no, deixi, comparèixer, davant, el, parlament, als, presos, polítics, és, una,...","[suprem, deixi, comparèixer, davant, parlament, presos, polítics, altra, ofensa, ciutadania, cat...","[suprem, deixi, comparèix, davant, parlament, preso, polític, altra, ofensa, ciutadania, catalun...","[suprem, deixi, comparèix, davant, parlament, preso, polític, altra, ofensa, ciutadania, catalun..."
1,bcomu,Un artículo a leer de @ainagallego sobre un tema importante “El cambio tecnológico en el trabajo...,Un artículo a leer de ainagallego sobre un tema importante “El cambio tecnológico en el trabajo ...,"[un, artículo, a, leer, de, ainagallego, sobre, un, tema, importante, el, cambio, tecnológico, e...","[artículo, leer, ainagallego, tema, importante, cambio, tecnológico, trabajo, amenaza, u, oportu...","[artículo, leer, ainagallego, tema, important, cambio, tecnológico, trabajo, amenaza, u, oportun...","[artículo, leer, ainagallego, tema, important, cambio, tecnológico, trabajo, amenaza, u, oportun..."
2,bcomu,El sentiment de desprotecció davant les incerteses que genera el canvi d’època i les polítiques ...,El sentiment de desprotecció davant les incerteses que genera el canvi d’època i les polítiques ...,"[el, sentiment, de, desprotecció, davant, les, incerteses, que, genera, el, canvi, d, època, i, ...","[sentiment, desprotecció, davant, incerteses, genera, canvi, època, polítiques, neoliberals, pro...","[sentiment, desprotecció, davant, incertes, genera, canvi, època, polítiqu, neoliber, provoca, a...","[sentiment, desprotecció, davant, incertes, genus, canvi, època, polítiqu, neoliber, provoca, ap..."
3,jxcat,"I cap d’ells te traça amb els micròfons i les grabadores, ni res a veure amb les clavegueres, ve...",I cap d’ells te traça amb els micròfons i les grabadores ni res a veure amb les clavegueres ves ...,"[i, cap, d, ells, te, traça, amb, els, micròfons, i, les, grabadores, ni, res, a, veure, amb, le...","[cap, traça, micròfons, grabadores, res, veure, clavegueres, ves, més, em, representen, exiliats...","[cap, traça, micròfon, grabador, re, veur, claveguer, ve, mé, em, representen, exiliat, preso, p...","[cap, traça, micròfon, grabador, re, veur, claveguer, ve, mé, em, representen, exiliat, preso, p..."
4,jxcat,El @sergimiquel s’ha acomiadat avui com a Secretari General de la @JNCatalunya amb un gran discu...,El sergimiquel s’ha acomiadat avui com a Secretari General de la JNCatalunya amb un gran discurs...,"[el, sergimiquel, s, ha, acomiadat, avui, com, a, secretari, general, de, la, jncatalunya, amb, ...","[sergimiquel, acomiadat, avui, secretari, general, jncatalunya, gran, discurs, llargament, ovaci...","[sergimiquel, acomiadat, avui, secretari, gener, jncatalunya, gran, discur, llargament, ovaciona...","[sergimiquel, acomiadat, avui, secretari, gener, jncatalunya, gran, discur, llargament, ovaciona..."


In [25]:
def clean_text(text):
    text = ' '.join(word for word in text)
    return text
df['final_text'] = df['text_lemmatized'].apply(lambda x: clean_text(x))
df.head()

,party,text,text_punct,text_tokenized,text_nonstop,text_stemmed,text_lemmatized,final_text
0,bcomu,Que el Suprem no deixi comparèixer davant el Parlament als presos polítics és una altra ofensa a...,Que el Suprem no deixi comparèixer davant el Parlament als presos polítics és una altra ofensa a...,"[que, el, suprem, no, deixi, comparèixer, davant, el, parlament, als, presos, polítics, és, una,...","[suprem, deixi, comparèixer, davant, parlament, presos, polítics, altra, ofensa, ciutadania, cat...","[suprem, deixi, comparèix, davant, parlament, preso, polític, altra, ofensa, ciutadania, catalun...","[suprem, deixi, comparèix, davant, parlament, preso, polític, altra, ofensa, ciutadania, catalun...",suprem deixi comparèix davant parlament preso polític altra ofensa ciutadania catalunya ho expli...
1,bcomu,Un artículo a leer de @ainagallego sobre un tema importante “El cambio tecnológico en el trabajo...,Un artículo a leer de ainagallego sobre un tema importante “El cambio tecnológico en el trabajo ...,"[un, artículo, a, leer, de, ainagallego, sobre, un, tema, importante, el, cambio, tecnológico, e...","[artículo, leer, ainagallego, tema, importante, cambio, tecnológico, trabajo, amenaza, u, oportu...","[artículo, leer, ainagallego, tema, important, cambio, tecnológico, trabajo, amenaza, u, oportun...","[artículo, leer, ainagallego, tema, important, cambio, tecnológico, trabajo, amenaza, u, oportun...",artículo leer ainagallego tema important cambio tecnológico trabajo amenaza u oportunidad
2,bcomu,El sentiment de desprotecció davant les incerteses que genera el canvi d’època i les polítiques ...,El sentiment de desprotecció davant les incerteses que genera el canvi d’època i les polítiques ...,"[el, sentiment, de, desprotecció, davant, les, incerteses, que, genera, el, canvi, d, època, i, ...","[sentiment, desprotecció, davant, incerteses, genera, canvi, època, polítiques, neoliberals, pro...","[sentiment, desprotecció, davant, incertes, genera, canvi, època, polítiqu, neoliber, provoca, a...","[sentiment, desprotecció, davant, incertes, genus, canvi, època, polítiqu, neoliber, provoca, ap...",sentiment desprotecció davant incertes genus canvi època polítiqu neoliber provoca aparició aque...
3,jxcat,"I cap d’ells te traça amb els micròfons i les grabadores, ni res a veure amb les clavegueres, ve...",I cap d’ells te traça amb els micròfons i les grabadores ni res a veure amb les clavegueres ves ...,"[i, cap, d, ells, te, traça, amb, els, micròfons, i, les, grabadores, ni, res, a, veure, amb, le...","[cap, traça, micròfons, grabadores, res, veure, clavegueres, ves, més, em, representen, exiliats...","[cap, traça, micròfon, grabador, re, veur, claveguer, ve, mé, em, representen, exiliat, preso, p...","[cap, traça, micròfon, grabador, re, veur, claveguer, ve, mé, em, representen, exiliat, preso, p...",cap traça micròfon grabador re veur claveguer ve mé em representen exiliat preso polític
4,jxcat,El @sergimiquel s’ha acomiadat avui com a Secretari General de la @JNCatalunya amb un gran discu...,El sergimiquel s’ha acomiadat avui com a Secretari General de la JNCatalunya amb un gran discurs...,"[el, sergimiquel, s, ha, acomiadat, avui, com, a, secretari, general, de, la, jncatalunya, amb, ...","[sergimiquel, acomiadat, avui, secretari, general, jncatalunya, gran, discurs, llargament, ovaci...","[sergimiquel, acomiadat, avui, secretari, gener, jncatalunya, gran, discur, llargament, ovaciona...","[sergimiquel, acomiadat, avui, secretari, gener, jncatalunya, gran, discur, llargament, ovaciona...",sergimiquel acomiadat avui secretari gener jncatalunya gran discur llargament ovacionat pel comp...


In [26]:
def gen_tend(party):
    if party == "pp" or party == "vallsbcn":
        tend = "uni"
    elif party == "bcomu"or party == "psc":
        tend = "neut"
    else:
        tend = "ind"
    return tend
df['tend'] = df['party'].apply(lambda x: gen_tend(x))
print(set(df["tend"].values))
df.head()

{'ind', 'uni', 'neut'}


,party,text,text_punct,text_tokenized,text_nonstop,text_stemmed,text_lemmatized,final_text,tend
0,bcomu,Que el Suprem no deixi comparèixer davant el Parlament als presos polítics és una altra ofensa a...,Que el Suprem no deixi comparèixer davant el Parlament als presos polítics és una altra ofensa a...,"[que, el, suprem, no, deixi, comparèixer, davant, el, parlament, als, presos, polítics, és, una,...","[suprem, deixi, comparèixer, davant, parlament, presos, polítics, altra, ofensa, ciutadania, cat...","[suprem, deixi, comparèix, davant, parlament, preso, polític, altra, ofensa, ciutadania, catalun...","[suprem, deixi, comparèix, davant, parlament, preso, polític, altra, ofensa, ciutadania, catalun...",suprem deixi comparèix davant parlament preso polític altra ofensa ciutadania catalunya ho expli...,neut
1,bcomu,Un artículo a leer de @ainagallego sobre un tema importante “El cambio tecnológico en el trabajo...,Un artículo a leer de ainagallego sobre un tema importante “El cambio tecnológico en el trabajo ...,"[un, artículo, a, leer, de, ainagallego, sobre, un, tema, importante, el, cambio, tecnológico, e...","[artículo, leer, ainagallego, tema, importante, cambio, tecnológico, trabajo, amenaza, u, oportu...","[artículo, leer, ainagallego, tema, important, cambio, tecnológico, trabajo, amenaza, u, oportun...","[artículo, leer, ainagallego, tema, important, cambio, tecnológico, trabajo, amenaza, u, oportun...",artículo leer ainagallego tema important cambio tecnológico trabajo amenaza u oportunidad,neut
2,bcomu,El sentiment de desprotecció davant les incerteses que genera el canvi d’època i les polítiques ...,El sentiment de desprotecció davant les incerteses que genera el canvi d’època i les polítiques ...,"[el, sentiment, de, desprotecció, davant, les, incerteses, que, genera, el, canvi, d, època, i, ...","[sentiment, desprotecció, davant, incerteses, genera, canvi, època, polítiques, neoliberals, pro...","[sentiment, desprotecció, davant, incertes, genera, canvi, època, polítiqu, neoliber, provoca, a...","[sentiment, desprotecció, davant, incertes, genus, canvi, època, polítiqu, neoliber, provoca, ap...",sentiment desprotecció davant incertes genus canvi època polítiqu neoliber provoca aparició aque...,neut
3,jxcat,"I cap d’ells te traça amb els micròfons i les grabadores, ni res a veure amb les clavegueres, ve...",I cap d’ells te traça amb els micròfons i les grabadores ni res a veure amb les clavegueres ves ...,"[i, cap, d, ells, te, traça, amb, els, micròfons, i, les, grabadores, ni, res, a, veure, amb, le...","[cap, traça, micròfons, grabadores, res, veure, clavegueres, ves, més, em, representen, exiliats...","[cap, traça, micròfon, grabador, re, veur, claveguer, ve, mé, em, representen, exiliat, preso, p...","[cap, traça, micròfon, grabador, re, veur, claveguer, ve, mé, em, representen, exiliat, preso, p...",cap traça micròfon grabador re veur claveguer ve mé em representen exiliat preso polític,ind
4,jxcat,El @sergimiquel s’ha acomiadat avui com a Secretari General de la @JNCatalunya amb un gran discu...,El sergimiquel s’ha acomiadat avui com a Secretari General de la JNCatalunya amb un gran discurs...,"[el, sergimiquel, s, ha, acomiadat, avui, com, a, secretari, general, de, la, jncatalunya, amb, ...","[sergimiquel, acomiadat, avui, secretari, general, jncatalunya, gran, discurs, llargament, ovaci...","[sergimiquel, acomiadat, avui, secretari, gener, jncatalunya, gran, discur, llargament, ovaciona...","[sergimiquel, acomiadat, avui, secretari, gener, jncatalunya, gran, discur, llargament, ovaciona...",sergimiquel acomiadat avui secretari gener jncatalunya gran discur llargament ovacionat pel comp...,ind


In [27]:
val_size = int(len(df) * 0)
x_train = df["final_text"][val_size:]
y_train = df["party"][val_size:]
x_val = df["final_text"][:val_size]
y_val = df["party"][:val_size]
print(x_train.shape,y_train.shape)
print(x_val.shape,y_val.shape)

(7219,) (7219,)
(0,) (0,)


In [28]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(min_df=0, lowercase=True,ngram_range=(1,1))
vectorizer.fit(x_train)
vectorizer.vocabulary_
x_train = vectorizer.transform(x_train).toarray()
vocab_size = np.shape(x_train)[1]
#x_val = vectorizer.transform(x_val).toarray()
print(vocab_size)
encoder = LabelBinarizer()
encoder.fit(y_train)
y_train = encoder.transform(y_train)
#y_val = encoder.transform(y_val)
print("Train shapes:")
print(x_train.shape,y_train.shape)
print("val shapes:")
#print(x_val.shape,y_val.shape)

26196
Train shapes:
(7219, 26196) (7219, 8)
val shapes:


In [29]:
# first we have to make sure to input data and params into the function
def political_model(x_train, y_train, x_val, y_val, params):

    # next we can build the model exactly like we would normally do it
    model = Sequential()
    model.add(Dense(params["first_neuron"], input_dim=x_train.shape[1],
                    activation=params['activation'],
                    kernel_initializer='normal'))
    
    model.add(Dropout(params['dropout']))
    
    model.add(Dense(params["second_neuron"],
                    activation=params['activation'],
                    kernel_initializer='normal'))
    
    model.add(Dropout(params['dropout']))

    # then we finish again with completely standard Keras way
    model.add(Dense(8, activation=params['last_activation'],
                    kernel_initializer='normal'))
    
    model.compile(loss=params['losses'],
                  # here we add a regulizer normalization function from Talos
                  optimizer=params['optimizer'],
                  metrics=['acc'])
    
    history = model.fit(x_train, y_train,
                        batch_size=params['batch_size'],
                        validation_data=[x_val, y_val],
                        epochs=params['epochs'],
                        verbose=0)
    
    # finally we have to make sure that history object and model are returned
    return history, model

In [30]:
p = {'first_neuron':[32,64,128,256,512,1024,2048],
     'second_neuron':[32,64,128,256,512,1024,2048],
     'batch_size': [10],
     'epochs': [5],
     'dropout': [0.1],
     'optimizer': ["Adam"],
     'losses': ["categorical_crossentropy"],
     'activation':["relu"],
     'last_activation': ["softmax"]}

In [31]:
import keras
import tensorflow as tf

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())
import talos as ta
x = x_train
y = y_train
t = ta.Scan(x,y,
            model=political_model,
            params=p)


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 14086303828106771485
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 9224895857
locality {
  bus_id: 1
  links {
  }
}
incarnation: 13744703829398177097
physical_device_desc: "device: 0, name: GeForce GTX 1080 Ti, pci bus id: 0000:01:00.0, compute capability: 6.1"
]


100%|██████████| 49/49 [27:18<00:00, 84.16s/it]


In [32]:
# accessing the results data frame
t.data.head()

# accessing epoch entropy values for each round

# access the summary details
t.details

experiment_name            062619184800
random_method          uniform_mersenne
reduction_method                   None
reduction_interval                   50
reduction_window                     20
reduction_threshold                 0.2
reduction_metric                val_acc
complete_time            06/26/19/19:15
x_shape                   (7219, 26196)
y_shape                       (7219, 8)
dtype: object

In [33]:
# use Scan object as input
r = ta.Reporting(t)

# access the dataframe with the results
r.data.head(-3)

,round_epochs,val_loss,val_acc,loss,acc,activation,batch_size,dropout,epochs,first_neuron,last_activation,losses,optimizer,second_neuron
0,5,1.846937,0.493536,0.260474,0.932120,relu,10,0.1,5,32,softmax,categorical_crossentropy,Adam,32
1,5,2.177223,0.480148,0.143143,0.968534,relu,10,0.1,5,32,softmax,categorical_crossentropy,Adam,64
2,5,1.740310,0.544321,0.062986,0.987136,relu,10,0.1,5,32,softmax,categorical_crossentropy,Adam,128
3,5,1.781457,0.544321,0.025337,0.997427,relu,10,0.1,5,32,softmax,categorical_crossentropy,Adam,256
4,5,1.772100,0.554940,0.020016,0.997823,relu,10,0.1,5,32,softmax,categorical_crossentropy,Adam,512
5,5,1.651493,0.568329,0.013234,0.998219,relu,10,0.1,5,32,softmax,categorical_crossentropy,Adam,1024
6,5,1.807089,0.572022,0.008066,0.998813,relu,10,0.1,5,32,softmax,categorical_crossentropy,Adam,2048
7,5,2.097432,0.489381,0.154600,0.968336,relu,10,0.1,5,64,softmax,categorical_crossentropy,Adam,32
8,5,2.081238,0.500923,0.053090,0.993667,relu,10,0.1,5,64,softmax,categorical_crossentropy,Adam,64
9,5,1.673881,0.555863,0.015847,0.998417,relu,10,0.1,5,64,softmax,categorical_crossentropy,Adam,128
